In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from TDSCoinbaseData import TDSCoinbaseData
from TDSTickGenerator import TDSTickGenerator
from TDSTransactionTracker import TDSTransactionTracker
import logging
logging.getLogger().setLevel(level=logging.ERROR)

In [3]:
# instantiate our data interface
cb_obj = TDSCoinbaseData(cache_path='data', notebook_logging=True)

# Set the start and end date
start_date = "20200312"
end_date = "20200312"

# List all of the products to be used in the strategy. Be sure to list all products you may use, these can't be updated later
products = ['BTC-USD', 'ETH-USD', 'BTC-EUR', 'ETH-EUR', 'LTC-USD', 'LTC-EUR']

# Instantiate our transaction tracker. As required, we start with 1.0 BTC
trans_tracker = TDSTransactionTracker(start_date, end_date, holdings={'BTC' : 1.0})

# Instantiate a tick generator
tick_gen = TDSTickGenerator(cb_obj, products, start_date, end_date, interval=60)
# print(trans_tracker.get_holdings())

tick = tick_gen.get_tick()
trans_tracker.make_trade(tick, 'BTC-USD', 'sell', 0.5)
print("DAY", start_date, ', INITIAL HOLDINGS : ')
# Check the updated holdings
print(trans_tracker.get_holdings())

DAY 20200312 , INITIAL HOLDINGS : 
{'BTC': 0.5, 'USD': 3971.3486820000003}


In [4]:
def trade_eth(tick, bu, eu, be, ee, has_usd):
        # Amount of ETH and BTC to trade at most due to volume restriction
        num_eth = min(tick.p.eth_usd.volume, tick.p.eth_eur.volume) / 2
        num_btc = num_eth * ee / be
        if (num_btc > min(tick.p.btc_usd.volume, tick.p.btc_eur.volume) / 2):
            num_btc = min(num_eth * ee / be, min(tick.p.btc_usd.volume, tick.p.btc_eur.volume) / 2)
            num_eth = num_btc * be / ee

        # Buy Bitcoin with USD
        trans_tracker.make_trade(tick, 'BTC-USD', 'buy', min(num_btc * bu, has_usd))
        print(trans_tracker.get_holdings())
        # Sell Bitcoin for EUR
        trans_tracker.make_trade(tick, 'BTC-EUR', 'sell', min(num_btc, trans_tracker.get_holdings().get("BTC")))
        print(trans_tracker.get_holdings())
        # Buy ETH with EUR
        trans_tracker.make_trade(tick, 'ETH-EUR', 'buy', min(num_eth * ee, trans_tracker.get_holdings().get("EUR")))
        print(trans_tracker.get_holdings())
        # Sell ETH with USD
        trans_tracker.make_trade(tick, 'ETH-USD', 'sell', trans_tracker.get_holdings().get("ETH"))
        print(trans_tracker.get_holdings())
        # Buy Bitcoin with USD
        trans_tracker.make_trade(tick, 'BTC-USD', 'buy', min(num_btc * bu, trans_tracker.get_holdings().get("USD")))
        print(trans_tracker.get_holdings())

In [5]:
def trade_ltc(tick, bu, lu, be, le, has_usd):
        # Amount of LTC and BTC to trade at most due to volume restriction
        num_ltc = min(tick.p.ltc_usd.volume, tick.p.ltc_eur.volume) / 2
        num_btc = num_ltc * le / be
        if (num_btc > min(tick.p.btc_usd.volume, tick.p.btc_eur.volume) / 2):
            num_btc = min(num_ltc * le / be, min(tick.p.btc_usd.volume, tick.p.btc_eur.volume) / 2)
            num_ltc = num_btc * be / le

        # Buy Bitcoin with USD
        trans_tracker.make_trade(tick, 'BTC-USD', 'buy', min(num_btc * bu, has_usd))
        print(trans_tracker.get_holdings())
        # Sell Bitcoin for EUR
        trans_tracker.make_trade(tick, 'BTC-EUR', 'sell', min(num_btc, trans_tracker.get_holdings().get("BTC")))
        print(trans_tracker.get_holdings())
        # Buy LTC with EUR
        trans_tracker.make_trade(tick, 'LTC-EUR', 'buy', min(num_ltc * le, trans_tracker.get_holdings().get("EUR")))
        print(trans_tracker.get_holdings())
        # Sell LTC with USD
        trans_tracker.make_trade(tick, 'LTC-USD', 'sell', trans_tracker.get_holdings().get("LTC"))
        print(trans_tracker.get_holdings())
        # Buy Bitcoin with USD
        trans_tracker.make_trade(tick, 'BTC-USD', 'buy', min(num_btc * bu, trans_tracker.get_holdings().get("USD")))
        print(trans_tracker.get_holdings())

In [6]:
def arbitrage(benchmark, tick):

    while tick is not None:
        bu = tick.p.btc_usd.close
        eu = tick.p.eth_usd.close
        lu = tick.p.ltc_usd.close
        be = tick.p.btc_eur.close
        ee = tick.p.eth_eur.close
        le = tick.p.ltc_eur.close
        
        has_usd = trans_tracker.get_holdings().get("USD")

        # Check exchange ratios for BTC - ETH and BTC - LTC
        be_ratio = (be * eu) / (ee * bu) * 100 - 100
        bl_ratio = (be * lu) / (le * bu) * 100 - 100
        
        # Check if it is an arbitrage opportunity for BTC - ETH or BTC - LTC
        if (be_ratio < benchmark and bl_ratio < benchmark):
            tick = tick_gen.get_tick()
            continue

        # Check which would guarantee larger profit
        if (be_ratio > bl_ratio):
            trade_eth(tick, bu, eu, be, ee, has_usd)
        else:
            trade_ltc(tick, bu, lu, be, le, has_usd)
        
        tick = tick_gen.get_tick()

In [8]:
arbitrage(1.4, tick)

In [9]:
display(trans_tracker.get_pct_change_per_day())
print(trans_tracker.get_sharpe_ratio())

,date,BTC,pct_diff
0,20200312,1.604147,0.604147


RISK FREE DAILY RETURN : 
0.00016438356164383562
ACTUAL DAILY RETURN : 
0.6041472377268959
EXCESS STD : 
nan
nan


In [19]:
def trade_ratio(b, e, l, tick, trans_tracker):
    btc = tick.p.btc_usd.close * trans_tracker.get_holdings().get("BTC")
    eth = tick.p.eth_usd.close * trans_tracker.get_holdings().get("ETH")
    ltc = tick.p.ltc_usd.close * trans_tracker.get_holdings().get("LTC")
    usd_val = btc + eth + ltc
    print(btc/usd_val, " ", eth/usd_val, " ", ltc/usd_val)
    
    btc_gap = (usd_val * b - btc) / tick.p.btc_usd.close
    eth_gap = (usd_val * e - eth) / tick.p.eth_usd.close
    ltc_gap = (usd_val * l - ltc) / tick.p.ltc_usd.close
    
    if (btc_gap > 0):
        trans_tracker.make_trade(tick, 'BTC-USD', 'sell', btc_gap / tick.p.btc_usd.close)
    if (eth_gap > 0):
        trans_tracker.make_trade(tick, 'ETH-USD', 'sell', eth_gap / tick.p.eth_usd.close)
    if (ltc_gap > 0):
        trans_tracker.make_trade(tick, 'LTC-USD', 'sell', ltc_gap / tick.p.ltc_usd.close)
        
    if (btc_gap < 0):
        trans_tracker.make_trade(tick, 'BTC-USD', 'buy', min(btc_gap, trans_tracker.get_holdings().get("USD")))
    if (eth_gap < 0):
        trans_tracker.make_trade(tick, 'ETH-USD', 'buy', min(eth_gap, trans_tracker.get_holdings().get("USD")))
    if (ltc_gap < 0):
        trans_tracker.make_trade(tick, 'LTC-USD', 'buy', min(ltc_gap, trans_tracker.get_holdings().get("USD")))

In [1]:
def get_coin_ratio(tick, holdings):
    btc = holdings.get("BTC") if "BTC" in holdings else 0
    eth = tick.p.eth_usd.close/tick.p.btc_usd.close * holdings.get("ETH") if "ETH" in holdings else 0
    ltc = tick.p.ltc_usd.close/tick.p.btc_usd.close * holdings.get("LTC") if "LTC" in holdings else 0
    usd = holdings.get("USD")/tick.p.btc_usd.close if "USD" in holdings else 0
    res = [btc, usd, eth, ltc]
    return res/sum(res)

In [23]:
get_coin_ratio(tick, trans_tracker)

[1.0, 0.0, 0.0]